# Packages

In [2]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import yfinance as yf
import yahoofinancials
from scipy.optimize import minimize
import seaborn as sn
from scipy.stats import anderson,kstest,shapiro, probplot

c:\Users\leona\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\leona\anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
c:\Users\leona\anaconda3\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


# Masalah.  
Misalkan portofolio saham dengan komposisi saham TLKM, BBNI, dan ASII. Akan dicari nilai return dengan variansi minimal dan juga dicari komposisi dari masing-masing saham.  

In [15]:
stock = ['TLKM.JK', 'BBNI.JK','ASII.JK', 'ANTM.JK', 'BMRI.JK','BBRI.JK', 'ARTO.JK', 'UNVR.JK', 'AMRT.JK', 'HRUM.JK',
        'BBCA.JK', 'INCO.JK', 'ADRO.JK', 'PTBA.JK', 'PGAS.JK']
#stock = ['TSLA', 'DIS', 'NKE', 'AAPL']
df = pd.DataFrame(columns=stock)
for i in stock :
    temp = yf.Ticker(i).history(period = '2y', interval = '1mo',actions=False)
    temp.dropna(inplace = True)
    temp['log_return'] = np.log(temp['Close']) - np.log(temp['Close'].shift(1))
    temp['log_return'].dropna(inplace = True)
    df[i] =temp['log_return'].dropna()

In [21]:
mu = df.mean().values
C = df.cov().values
def objective(weights): 
    weights = np.array(weights)
    return weights.dot(C).dot(weights.T)
# The constraints
cons = (# The weights must sum up to one.
        {"type":"eq", "fun": lambda x: np.sum(x)-1}, 
        # This constraints says that the inequalities (ineq) must be non-negative.# The expected daily return of our portfolio and we want to be at greater than 0.002352
        {"type": "ineq", "fun": lambda x: np.sum(mu*x)-0.06}

        )
# Every stock can get any weight from 0 to 1
bounds = tuple((0,1) for x in range(mu.shape[0]))
# Initialize the weights with an even split
# In out case each stock will have 10% at the beginning
guess = [1./mu.shape[0] for x in range(mu.shape[0])]
optimized_results = minimize(objective, guess, method = "SLSQP", bounds=bounds, constraints=cons)
x_ = optimized_results.x

In [22]:
np.sum(np.round(x_,5))

1.0

In [23]:
port_ = dict(zip(stock,np.round(x_,5)))
port = dict()
for (key,value) in port_.items():
    if value >0 : 
        port[key] = value
port

{'BBNI.JK': 0.27961,
 'UNVR.JK': 0.04356,
 'INCO.JK': 0.13001,
 'ADRO.JK': 0.40135,
 'PGAS.JK': 0.14547}

In [24]:
start = 2e7 
budget_ = dict()
for (key,value) in port.items():
    if value >0 : 
        budget_[key] = value*start
budget_

{'BBNI.JK': 5592200.000000001,
 'UNVR.JK': 871200.0,
 'INCO.JK': 2600199.9999999995,
 'ADRO.JK': 8027000.0,
 'PGAS.JK': 2909399.9999999995}

In [25]:
budget = pd.DataFrame.from_dict(data= budget_, orient= 'index').T
budget.rename(index={0: 'Total Spread'}, inplace= True)
budget

,BBNI.JK,UNVR.JK,INCO.JK,ADRO.JK,PGAS.JK
Total Spread,5592200.0,871200.0,2600200.0,8027000.0,2909400.0


In [26]:
expense = pd.DataFrame(columns=port.keys())
for item in port.keys(): 
    temp = yf.Ticker(item).history(period = '1d', interval = '1mo',actions=False)
    expense[item] = temp['Close']*100 #1 lot
expense.rename(index={expense.index[0]: str('Closing ' + str(expense.index[0]).split(' ')[0])}, inplace= True)
expense

,BBNI.JK,UNVR.JK,INCO.JK,ADRO.JK,PGAS.JK
Date,,,,,
Closing 2022-05-17,845000.0,480000.0,727500.0,322000.0,156000.0


In [27]:
qty = pd.concat([expense,budget])
temp_ = pd.DataFrame(np.round(qty.iloc[1] / qty.iloc[0],0)).T
temp_.rename(index={0: 'Total Lot'}, inplace= True)
qty = pd.concat([qty, temp_])
spend = pd.DataFrame(np.round(qty.iloc[2] * qty.iloc[0],0)).T
finale = pd.concat([qty, spend])
finale.rename(index={0: 'Total Spending'}, inplace= True)
finale

,BBNI.JK,UNVR.JK,INCO.JK,ADRO.JK,PGAS.JK
Closing 2022-05-17,845000.0,480000.0,727500.0,322000.0,156000.0
Total Spread,5592200.0,871200.0,2600200.0,8027000.0,2909400.0
Total Lot,7.0,2.0,4.0,25.0,19.0
Total Spending,5915000.0,960000.0,2910000.0,8050000.0,2964000.0


In [28]:
finale.sum(axis = 1)

Closing 2022-05-17     2530500.0
Total Spread          20000000.0
Total Lot                   57.0
Total Spending        20799000.0
dtype: float64

In [29]:
print(f'return :', x_.dot(mu)*100, '%')
print(f'risk :', x_.dot(C).dot(x_)*100, '%')
print(f'Total Money :',finale.sum(axis=1)[-1])
print(f'Total Idle Money :', start - finale.sum(axis=1)[-1])

return : 5.999999999643227 %
risk : 0.3184243245154742 %
Total Money : 20799000.0
Total Idle Money : -799000.0


In [30]:
finale.to_csv(str('IHSG '+finale.index[0].split(' ')[1]+'.csv'))

# Testing Profit

In [163]:
year = 2010
ds = str(year-1)+'-12-01'
de = str(year+2)+'-12-01'

stock = ['TLKM.JK', 'BBNI.JK','ASII.JK', 'ANTM.JK', 'BMRI.JK','BBRI.JK', 'ARTO.JK', 'UNVR.JK', 'AMRT.JK', 'HRUM.JK',
        'BBCA.JK', 'INCO.JK', 'ADRO.JK', 'PTBA.JK', 'PGAS.JK']
#stock = ['TSLA', 'DIS', 'NKE', 'AAPL']
df = pd.DataFrame(columns=stock)
for i in stock :
    temp = yf.Ticker(i).history(period = '2y', start = ds ,end =de ,interval = '1mo',actions=False)
    temp.dropna(inplace = True)
    temp['log_return'] = np.log(temp['Close']) - np.log(temp['Close'].shift(1))
    temp['log_return'].dropna(inplace = True)
    df[i] =temp['log_return'].dropna()
df.dropna(axis = 1, inplace = True)

- ARTO.JK: Data doesn't exist for startDate = 1259600400, endDate = 1354294800


In [171]:
mu = df.mean().values
C = df.cov().values

def objective(weights): 
    weights = np.array(weights)
    return weights.dot(C).dot(weights.T)
# The constraints
cons = (# The weights must sum up to one.
        {"type":"eq", "fun": lambda x: np.sum(x)-1}, 
        # This constraints says that the inequalities (ineq) must be non-negative.# The expected daily return of our portfolio and we want to be at greater than 0.002352
        #{"type": "ineq", "fun": lambda x: np.sum(mu*x)-0.06}

        )
# Every stock can get any weight from 0 to 1
bounds = tuple((0,1) for x in range(mu.shape[0]))
# Initialize the weights with an even split
# In out case each stock will have 10% at the beginning
guess = [1./mu.shape[0] for x in range(mu.shape[0])]
optimized_results = minimize(objective, guess, method = "SLSQP", bounds=bounds, constraints=cons)
x_ = optimized_results.x

In [172]:
print(np.sum(np.round(x_,5)))
print(f'return :', x_.dot(mu)*100, '%')
print(f'risk :', x_.dot(C).dot(x_)*100, '%')
print(f'Total Money :',finale.sum(axis=1)[-1])
print(f'Total Idle Money :', start - finale.sum(axis=1)[-1])

1.0
return : 0.39857219698860075 %
risk : 0.016070674275375892 %
Total Money : 20015931.0
Total Idle Money : -15931.0


In [173]:
port_ = dict(zip(df.columns.values,np.round(x_,5)))
port = dict()
for (key,value) in port_.items():
    if value >0 : 
        port[key] = value
start = 2e7 
budget_ = dict()
for (key,value) in port.items():
    if value >0 : 
        budget_[key] = value*start
budget = pd.DataFrame.from_dict(data= budget_, orient= 'index').T
budget.rename(index={0: 'Total Spread'}, inplace= True)
expense = pd.DataFrame(columns=port.keys())

bs = str(year+2)+'-12-31'
be = str(year+3)+'-01-01'
for item in port.keys(): 
    temp = yf.Ticker(item).history(period = '1d', start = bs, end = be, interval = '1mo',actions=False)
    expense[item] = temp['Close']*100 #1 lot
expense.rename(index={expense.index[0]: str('Closing ' + str(expense.index[0]).split(' ')[0])}, inplace= True)
qty = pd.concat([expense,budget])
temp_ = pd.DataFrame(np.round(qty.iloc[1] / qty.iloc[0],0)).T
temp_.rename(index={0: 'Total Lot'}, inplace= True)
qty = pd.concat([qty, temp_])
spend = pd.DataFrame(np.round(qty.iloc[2] * qty.iloc[0],0)).T
finale = pd.concat([qty, spend])
finale.rename(index={0: 'Total Spending'}, inplace= True)
finale

,TLKM.JK,BBNI.JK
Closing 2013-01-01,155220.88623,1.390727e+05
Total Spread,958800.00000,1.904120e+07
Total Lot,6.00000,1.370000e+02
Total Spending,931325.00000,1.905296e+07


In [174]:
buy = finale.columns.values
bs = str(year+2)+'-12-31'
be = str(year+4)+'-01-01'
profit_test = pd.DataFrame()
for item in buy:
    temp = yf.Ticker(item).history(period = '1y', start = bs, end = be, interval = '1mo',actions=False)
    temp.dropna(inplace = True)
    temp['buy_'+item] = finale[item].loc['Total Lot'] * temp['Open'] * 100
    temp['sell_'+item] = finale[item].loc['Total Lot'] * temp['Close'] * 100
    profit_test['buy_'+item] = temp['buy_'+item].dropna()
    profit_test['sell_'+item] = temp['sell_'+item].dropna()
    profit_test['return_'+item] =  profit_test['sell_'+item].iloc[1:] - profit_test['buy_'+item].iloc[:-1]
profit_test.dropna(inplace = True)
profit_test


,buy_TLKM.JK,sell_TLKM.JK,return_TLKM.JK,buy_BBNI.JK,sell_BBNI.JK,return_BBNI.JK
Date,,,,,,
2013-02-01,9.265246e+05,1.032139e+06,105614.202171,1.905297e+07,1.905297e+07,0.000000e+00
2013-03-01,1.041740e+06,1.056142e+06,14401.939586,1.905297e+07,1.905297e+07,0.000000e+00
2013-04-01,1.056142e+06,1.123351e+06,67209.044471,1.905296e+07,5.501925e+07,3.596629e+07
2013-05-01,1.123351e+06,1.060943e+06,-62408.401310,5.621018e+07,5.074531e+07,-5.464879e+06
2013-06-01,1.071542e+06,1.086023e+06,14480.302734,5.100552e+07,4.475995e+07,-6.245574e+06
2013-07-01,1.061889e+06,1.148771e+06,86881.816037,4.502019e+07,4.449972e+07,-5.204646e+05
2013-08-01,1.153598e+06,1.061889e+06,-91708.592418,4.475996e+07,4.007577e+07,-4.684181e+06
2013-09-01,1.061889e+06,1.013621e+06,-48267.679269,3.955531e+07,4.241786e+07,2.862555e+06
2013-10-01,1.013621e+06,1.134290e+06,120669.192050,4.267810e+07,4.996461e+07,7.286505e+06


In [188]:
cols = profit_test.columns
finale_pt = pd.DataFrame()
tspend = 0
tprofit = 0 
for k in range (0,profit_test.shape[1], 3):
    tspend += profit_test[cols[k]]
    tprofit += profit_test[cols[k+2]]
finale_pt['Total Spending'] = tspend
finale_pt['Total Profit'] = tprofit
finale_pt['Total Return'] = finale_pt['Total Profit']/finale_pt['Total Spending']
finale_pt

,Total Spending,Total Profit,Total Return
Date,,,
2013-02-01,1.997949e+07,1.056142e+05,0.005286
2013-03-01,2.009471e+07,1.440194e+04,0.000717
2013-04-01,2.010911e+07,3.603350e+07,1.791900
2013-05-01,5.733354e+07,-5.527287e+06,-0.096406
2013-06-01,5.207707e+07,-6.231094e+06,-0.119651
2013-07-01,4.608208e+07,-4.335828e+05,-0.009409
2013-08-01,4.591355e+07,-4.775890e+06,-0.104019
2013-09-01,4.061720e+07,2.814287e+06,0.069288
2013-10-01,4.369172e+07,7.407174e+06,0.169533


In [189]:
print(f'Keuntungan dalam satu tahun pada tahun {year+3} sebesar:', finale_pt['Total Return'].mean()*100,'%')

Keuntungan dalam satu tahun pada tahun 2013 sebesar: 13.993236726991984 %


# Loop

In [190]:
year = 2010
for n in range(10):
    year += n
    ds = str(year-1)+'-12-01'
    de = str(year+2)+'-12-01'

    stock = ['TLKM.JK', 'BBNI.JK','ASII.JK', 'ANTM.JK', 'BMRI.JK','BBRI.JK', 'ARTO.JK', 'UNVR.JK', 'AMRT.JK', 'HRUM.JK',
            'BBCA.JK', 'INCO.JK', 'ADRO.JK', 'PTBA.JK', 'PGAS.JK']
    #stock = ['TSLA', 'DIS', 'NKE', 'AAPL']
    df = pd.DataFrame(columns=stock)
    for i in stock :
        temp = yf.Ticker(i).history(period = '2y', start = ds ,end =de ,interval = '1mo',actions=False)
        temp.dropna(inplace = True)
        temp['log_return'] = np.log(temp['Close']) - np.log(temp['Close'].shift(1))
        temp['log_return'].dropna(inplace = True)
        df[i] =temp['log_return'].dropna()
    df.dropna(axis = 1, inplace = True)

    mu = df.mean().values
    C = df.cov().values

    def objective(weights): 
        weights = np.array(weights)
        return weights.dot(C).dot(weights.T)
    # The constraints
    cons = (# The weights must sum up to one.
            {"type":"eq", "fun": lambda x: np.sum(x)-1}, 
            # This constraints says that the inequalities (ineq) must be non-negative.# The expected daily return of our portfolio and we want to be at greater than 0.002352
            #{"type": "ineq", "fun": lambda x: np.sum(mu*x)-0.06}

            )
    # Every stock can get any weight from 0 to 1
    bounds = tuple((0,1) for x in range(mu.shape[0]))
    # Initialize the weights with an even split
    # In out case each stock will have 10% at the beginning
    guess = [1./mu.shape[0] for x in range(mu.shape[0])]
    optimized_results = minimize(objective, guess, method = "SLSQP", bounds=bounds, constraints=cons)
    x_ = optimized_results.x
    print(np.sum(np.round(x_,5)))
    print(f'return :', x_.dot(mu)*100, '%')
    print(f'risk :', x_.dot(C).dot(x_)*100, '%')
    print(f'Total Money :',finale.sum(axis=1)[-1])
    print(f'Total Idle Money :', start - finale.sum(axis=1)[-1])
    port_ = dict(zip(df.columns.values,np.round(x_,5)))
    port = dict()
    for (key,value) in port_.items():
        if value >0 : 
            port[key] = value
    start = 2e7 
    budget_ = dict()
    for (key,value) in port.items():
        if value >0 : 
            budget_[key] = value*start
    budget = pd.DataFrame.from_dict(data= budget_, orient= 'index').T
    budget.rename(index={0: 'Total Spread'}, inplace= True)
    expense = pd.DataFrame(columns=port.keys())

    bs = str(year+2)+'-12-31'
    be = str(year+3)+'-01-01'
    for item in port.keys(): 
        temp = yf.Ticker(item).history(period = '1d', start = bs, end = be, interval = '1mo',actions=False)
        expense[item] = temp['Close']*100 #1 lot
    expense.rename(index={expense.index[0]: str('Closing ' + str(expense.index[0]).split(' ')[0])}, inplace= True)
    qty = pd.concat([expense,budget])
    temp_ = pd.DataFrame(np.round(qty.iloc[1] / qty.iloc[0],0)).T
    temp_.rename(index={0: 'Total Lot'}, inplace= True)
    qty = pd.concat([qty, temp_])
    spend = pd.DataFrame(np.round(qty.iloc[2] * qty.iloc[0],0)).T
    finale = pd.concat([qty, spend])
    finale.rename(index={0: 'Total Spending'}, inplace= True)
    finale
    buy = finale.columns.values
    bs = str(year+2)+'-12-31'
    be = str(year+4)+'-01-01'
    profit_test = pd.DataFrame()
    for item in buy:
        temp = yf.Ticker(item).history(period = '1y', start = bs, end = be, interval = '1mo',actions=False)
        temp.dropna(inplace = True)
        temp['buy_'+item] = finale[item].loc['Total Lot'] * temp['Open'] * 100
        temp['sell_'+item] = finale[item].loc['Total Lot'] * temp['Close'] * 100
        profit_test['buy_'+item] = temp['buy_'+item].dropna()
        profit_test['sell_'+item] = temp['sell_'+item].dropna()
        profit_test['return_'+item] =  profit_test['sell_'+item].iloc[1:] - profit_test['buy_'+item].iloc[:-1]
    profit_test.dropna(inplace = True)
    cols = profit_test.columns
    finale_pt = pd.DataFrame()
    tspend = 0
    tprofit = 0 
    for k in range (0,profit_test.shape[1], 3):
        tspend += profit_test[cols[k]]
        tprofit += profit_test[cols[k+2]]
    finale_pt['Total Spending'] = tspend
    finale_pt['Total Profit'] = tprofit
    finale_pt['Total Return'] = finale_pt['Total Profit']/finale_pt['Total Spending']
    finale_pt.to_csv('Profit Test' +str(year)+'.csv')
    print(f'Keuntungan dalam satu tahun pada tahun {year+3} sebesar:', profit_test['total_return'].mean()/start*100)